In [70]:
import cv2
import numpy as np

MIN_HEIGHT = 100
MIN_WIDTH = 100
MIN_BRIGHTNESS = 50


def check_image_quality(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found or could not be loaded: {image_path}")
            return False


        if image.shape[0] < MIN_HEIGHT or image.shape[1] < MIN_WIDTH:
            return False

        # Example: check if the image is too dar
        if np.mean(image) < MIN_BRIGHTNESS:
            return False

        return True
    except Exception as e:
        print(f"Error checking image quality: {e}")
        return False

# Usage
is_good_quality = check_image_quality('data/William/images/William_04f57bf9-74b4-4269-adcc-6d7096345f86.png')
is_good_quality

True

In [71]:
def check_eye_region(image, eye_box):
    """
    Check if the eye region defined by the bounding box is valid.
    Args:
        image: The image in which the eye region is located.
        eye_box: The bounding box of the eye region in the format [x, y, width, height].
    
    Returns:
        bool: True if the eye region is valid, False otherwise.
    """
    x, y, width, height = eye_box
    if x < 0 or y < 0 or width <= 0 or height <= 0:
        return False
    if x + width > image.shape[1] or y + height > image.shape[0]:
        return False
    return True


In [105]:
import numpy as np

def compare_with_perfect_user(participant_eye_data, perfect_user_eye_data):
    """
    Compare the participant's eye data with the perfect user's eye data.
    Args:
        participant_eye_data (list): The eye box and pupil data for the participant.
        perfect_user_eye_data (list): The eye box and pupil data for the perfect user.

    Returns:
        bool: True if the participant data is within an acceptable range of the perfect user data.
    """
    # Normalize data if needed (example, you may need to adjust this)
    # participant_eye_data = normalize(participant_eye_data)
    # perfect_user_eye_data = normalize(perfect_user_eye_data)

    # Calculate standard deviation for the perfect user data
    std_deviation = np.std(perfect_user_eye_data)

    # Calculate deviation for each data point and compare with a dynamic threshold
    for p_data, pu_data in zip(participant_eye_data, perfect_user_eye_data):
        deviation = abs(p_data - pu_data)

        threshold = std_deviation * 0.5

        if deviation > threshold:
            return False

    return True


In [107]:
def extract_key_from_filename(filename):
    # This is where you extract the matching part of the filename.
    # It's an example, you'll need to define how to extract the matching key.
    return filename.split('_')[1]


In [108]:
def normalize_eye_data(x, y, screen_width, screen_height):
    return x / screen_width, y / screen_height


In [109]:
def load_user_data(user_data_path, screen_width, screen_height):
    user_data = {}

    try:
        with open(user_data_path, 'r') as f:
            for line in f:
                parts = line.strip().split(',')
                if len(parts) < 15:
                    continue

                raw_x, raw_y = parts[1:3]
                calibration_points = normalize_eye_data(int(raw_x), int(raw_y), screen_width, screen_height)
                eye_box_pupil_data = list(map(float, parts[3:15]))
                
                user_data[calibration_points] = eye_box_pupil_data
    except Exception as e:
        print(f"Error loading user data: {e}")

    return user_data


In [113]:
partisipant_data = load_user_data('data/Shaq/calibration_data.csv', 1280,720)
perfect_user_data = load_user_data('data/William/calibration_data.csv', 1707,960)

def is_within_tolerance(point1, point2, tolerance=0.01):
    """
    Check if point1 is within a certain tolerance of point2.
    Args:
        point1 (tuple): The first point (x, y).
        point2 (tuple): The second point (x, y).
        tolerance (float): The acceptable tolerance.

    Returns:
        bool: True if point1 is within the tolerance of point2.
    """
    return abs(point1[0] - point2[0]) <= tolerance and abs(point1[1] - point2[1]) <= tolerance

# Compare the data with tolerance
for calibration_points, eye_box_pupil_data in partisipant_data.items():
    found_match = False
    for perfect_calibration_points in perfect_user_data:
        if is_within_tolerance(calibration_points, perfect_calibration_points):
            perfect_user_eye_box_pupil_data = perfect_user_data[perfect_calibration_points]
            if compare_with_perfect_user(eye_box_pupil_data, perfect_user_eye_box_pupil_data):
                print(f"Data for calibration points {calibration_points} is valid")
            else:
                print(f"Data for calibration points {calibration_points} is invalid")
            found_match = True
            break

    if not found_match:
        print(f"Calibration points {calibration_points} are not in the perfect user data within tolerance")


Data for calibration points (0.015625, 0.027777777777777776) is invalid
Data for calibration points (0.015625, 0.9722222222222222) is invalid
Data for calibration points (0.984375, 0.027777777777777776) is invalid
Data for calibration points (0.984375, 0.9722222222222222) is invalid
Data for calibration points (0.13671875, 0.14583333333333334) is invalid
Data for calibration points (0.13671875, 0.2638888888888889) is invalid
Data for calibration points (0.13671875, 0.3819444444444444) is invalid
Data for calibration points (0.13671875, 0.5) is invalid
Data for calibration points (0.13671875, 0.6180555555555556) is invalid
Data for calibration points (0.13671875, 0.7361111111111112) is invalid
Data for calibration points (0.13671875, 0.8541666666666666) is invalid
Data for calibration points (0.2578125, 0.14583333333333334) is invalid
Data for calibration points (0.2578125, 0.2638888888888889) is invalid
Data for calibration points (0.2578125, 0.3819444444444444) is invalid
Data for cal